## Install prerequisites

In [ ]:
!pip install tabulate json

In [ ]:
# Use this command to install the current directory as a python package in editable mode.
# Run the command from the same directory as the pyproject.toml file.
# pip install -e .

In [1]:
# import stuff
import spynns
import tabulate
import json
# from IPython.core.debugger import Pdb

## Setup the network
`spynns.Node(net)` creates a node and returns it for you to manipulate.
While it registers the node to the network, you're expected to maintain
your own references to the node so that you can control it later.

As you can see, you can do this by storing every node you create in a list.

The network is a directed graph, and to give it structure, you can connect the spiking output of one node to another using a connection in the form of a `spynns.Edge`.
You can create a connection by using `net.connect(parent, child, weight, delay)`
Or with `node.add_output(child, weight, delay)`.

In [2]:
net = spynns.Network()

inputs = [spynns.Node(net, threshold=10)]
outputs = [spynns.Node(net, threshold=10)]

# connect the output of the first node to the second node
inputs[0].add_output(outputs[0], weight=100, delay=1.0)
# connect the output of the second node to the first node, creating a loop
outputs[0].add_output(inputs[0], weight=11, delay=0.2)

nodes = inputs + outputs

Here's how you can apply a spike to a node:  
```node.apply_spike(amplitude, delay)```
And you can run the network for any amount of time:  
```
net.runfor(t: float)  # runs the network for `t` seconds
net.runtil(t: float)  # runs until the network has simulated `t` seconds total in its lifetime
```

In [8]:
inputs[0].apply_spike(11)

In [4]:
net.runfor(9.1)

In [5]:
# Show network output/state
counts = spynns.fires(nodes)
vectors = spynns.vectors(nodes)
lastfires = spynns.lastfires(nodes)
charges = spynns.charges(nodes)
# histories = [node.history for node in nodes]

data = zip(
    range(len(counts)),
    charges,
    counts,
    lastfires,
    vectors,
    # histories,
)
data = [[
    'id',
    'charge',
    'fires',
    't_lastfire',
    't_fires',
    # 'histories',
]] + list(data)
tabulate.tabulate(data, tablefmt='html')

id,charge,fires,t_lastfire,t_fires
0,0,8,8.4,"[0, 1.2, 2.4000000000000004, 3.6000000000000005, 4.800000000000001, 6.000000000000001, 7.200000000000001, 8.4]"
1,0,7,8.200000000000001,"[1.0, 2.2, 3.4000000000000004, 4.6000000000000005, 5.800000000000001, 7.000000000000001, 8.200000000000001]"


In [6]:
net.time  # the network time

9.0

In [7]:
net.queue  # spikes that will get processed in the future

[Spike(amplitude=100, time=9.4, source=Node at 19a4a653790 w/ Threshold: 10, Leak: None, refract: 0.1, connections: ['<Edge w: 100, d: 1.0, dest: 3950>'], destination=Node at 19a4a653950 w/ Threshold: 10, Leak: None, refract: 0.1, connections: ['<Edge w: 11, d: 0.2, dest: 3790>'])]

## Importing/exporting networks
This is not working yet. I based this code off of an importer/exporter I wrote for Tennlab's caspian networks, and I haven't fully made it work here yet.
The code is in the `spynns.network` module in the `network_from_json()` and `to_tennlab()` functions.
`spynns.feedforward` has also not been fully converted over yet either.

Here's how all that stuff worked in my old implementation:

In [ ]:
# import from json file
with open("../experiment_tenn2_tenngineered-milling.json") as f:
    j = json.loads(f.read())

nodes, inputs, outputs = casPYan.network_from_json(j)

In [ ]:
# export to json
d: dict = casPYan.to_tennlab(pnodes, inputs, outputs)
j = json.dumps(d)

## Plasticity/Backprop???

I'm still theorycrafting how to do this.